In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as sum_col, avg, when, to_date, year, month, dayofweek, lit, lag, count, row_number
from pyspark.sql.functions import min as spark_min, max as spark_max, avg as spark_avg
from pyspark.sql.window import Window
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

spark = SparkSession.builder \
    .appName("COVID19_Brasil_Analysis") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "5g") \
    .getOrCreate()


display("Spark Session iniciada com sucesso!")


# %%
from pyspark.sql import functions as F

# %%
caminho_do_parquet_tratado = "dataset\df_covid_tratado.parquet"
df_base_para_analise = spark.read.parquet(caminho_do_parquet_tratado)
df_base_para_analise.cache()
print("DataFrame base para análise carregado com sucesso!")
df_base_para_analise.printSchema()

# %%
df_base_para_analise.show(5)

# %% [markdown]
# interior x metrópole (óbitos)

# %%
df_agg_tipo = df_base_para_analise.groupBy("data", "interior/metropolitana") \
    .agg(
        F.sum("casosNovos").alias("soma_casos"),
        F.sum("obitosNovos").alias("soma_obitos")
    )

# %%
df_tipo_pd = df_agg_tipo.toPandas()

# Mapear os valores para nomes legíveis
df_tipo_pd["tipoLocal"] = df_tipo_pd["interior/metropolitana"].map({0: "Metropolitana", 1: "Interior"})

# %%
pivot_casos = df_tipo_pd.pivot(index="data", columns="tipoLocal", values="soma_casos")

pivot_casos.plot(figsize=(12, 6), title="Casos Novos - Metrópoles vs. Interior")
plt.xlabel("Data")
plt.ylabel("Casos Novos")
plt.grid(True)
plt.tight_layout()
plt.show()

# %%
pivot_obitos = df_tipo_pd.pivot(index="data", columns="tipoLocal", values="soma_obitos")

pivot_obitos.plot(figsize=(12, 6), title="Óbitos Novos - Metrópoles vs. Interior", color=["darkred", "orange"])
plt.xlabel("Data")
plt.ylabel("Óbitos Novos")
plt.grid(True)
plt.tight_layout()
plt.show()

# %% [markdown]
# região de saúde e ano

# %%
df_agg_regiao = df_base_para_analise.groupBy("nomeRegiaoSaude", "ano_data") \
    .agg(
        F.sum("casosNovos").alias("soma_casos"),
        F.sum("obitosNovos").alias("soma_obitos")
    )

# %%
df_regiao_pd = df_agg_regiao.toPandas()

# %%
df_top2021 = df_regiao_pd[df_regiao_pd["ano_data"] == 2021] \
    .sort_values(by="soma_obitos", ascending=False).head(10)

plt.figure(figsize=(12, 6))
plt.barh(df_top2021["nomeRegiaoSaude"], df_top2021["soma_obitos"], color='crimson')
plt.title("Top 10 Regiões de Saúde com mais Óbitos - 2021")
plt.xlabel("Óbitos")
plt.ylabel("Região de Saúde")
plt.gca().invert_yaxis()  # Coloca a maior barra no topo
plt.tight_layout()
plt.show()

# %%
df_top2020 = df_regiao_pd[df_regiao_pd["ano_data"] == 2020] \
    .sort_values(by="soma_obitos", ascending=False).head(10)

# %%
anos = sorted(df_regiao_pd["ano_data"].unique())

for ano in anos:
    df_top = df_regiao_pd[df_regiao_pd["ano_data"] == ano] \
        .sort_values(by="soma_obitos", ascending=False).head(10)
    
    plt.figure(figsize=(12, 6))
    plt.barh(df_top["nomeRegiaoSaude"], df_top["soma_obitos"], color='crimson')
    plt.title(f"Top 10 Regiões de Saúde com mais Óbitos - {ano}")
    plt.xlabel("Óbitos")
    plt.ylabel("Região de Saúde")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()